In [ ]:
# todo: check the model cfg: augmentations, policy

In [32]:
from torch import nn
from utils import parse_cfg
from darknet import EmptyLayer

In [11]:
layers_info = parse_cfg('cfg/yolov3.cfg')

In [ ]:
def make_modules(layers_info):
    pass
#     return modules


In [ ]:
# the first element is not a layer but the network info (lr, batchsize,  ...)
net_info = layers_info[0]
# init. the modulelist instead of a list to add all parameters to nn.Module
layer_list = nn.ModuleList()

for i, layer_info in layers_info[1:]:
    # we initialize sequential as a layer may have conv, bn, and activation
    layer = nn.Sequential()
    # cache the # of filters as we will need them in Conv2d
    # it starts with 3 because of RGB (3 channel) input
    filters_cache = [3]
    name = layer_info['name'] # conv, upsample, route, shortcut, yolo
    
    if name == 'convolutional':
        # extract arguments for the layer
        in_filters = filters_cache[-1]
        out_filters = int(layer_info['filters'])
        kernel_size = int(layer_info['size'])
        pad = (kernel_size - 1) // 2 if int(module_def['pad']) else 0
        stride = int(layer_info['stride'])
        
        # make conv module and add it to the sequential
        conv = nn.Conv2d(in_filters, out_filters, kernel_size, stride, pad)
        layer.add_module('conv_{}'.format(i), conv)
        
        # some layers doesn't have BN
        try:
            layer_info['batch_normalize']
            layer.add_module('bn_' + i, nn.BatchNorm2d(out_filters))
            
        except KeyError:
            print('this message should be printed 3ish times')
            pass
        
        # activation. if 'linear': no activation
        if layer_info['activation'] == 'leaky':
            layer.add_module('leaky_' + i, nn.LeakyReLU(0.1))
        
        # add the number of filters to filters_cache
        filters_cache.append(out_filters)
    
    elif name == 'upsample':
        # extract arguments for the layer
        stride = int(layer_info['stride'])
        layer.add_module('upsample_' + i, nn.Upsample(scale_factor=stride, mode='bilinear'))
        
    # here we need to deal only with the number of filters 
    elif name == 'route':
        # route can have one or two sources, make them to be ints
        routes = [int(route) for route in layer_info['layers'].split(',')]
        
            
        # add the number of filters to filters_cache
        filters_cache.append(out_filters)
        
    elif name == 'shortcut':
        # todo
        
    
    # append the layer to the modulelist
    layer_list.append(layer)

In [34]:
a = '-1'
a.split(',')

['-1']

In [13]:
layers_info[1:]

[{'activation': 'leaky',
  'batch_normalize': '1',
  'filters': '32',
  'name': 'convolutional',
  'pad': '1',
  'size': '3',
  'stride': '1'},
 {'activation': 'leaky',
  'batch_normalize': '1',
  'filters': '64',
  'name': 'convolutional',
  'pad': '1',
  'size': '3',
  'stride': '2'},
 {'activation': 'leaky',
  'batch_normalize': '1',
  'filters': '32',
  'name': 'convolutional',
  'pad': '1',
  'size': '1',
  'stride': '1'},
 {'activation': 'leaky',
  'batch_normalize': '1',
  'filters': '64',
  'name': 'convolutional',
  'pad': '1',
  'size': '3',
  'stride': '1'},
 {'activation': 'linear', 'from': '-3', 'name': 'shortcut'},
 {'activation': 'leaky',
  'batch_normalize': '1',
  'filters': '128',
  'name': 'convolutional',
  'pad': '1',
  'size': '3',
  'stride': '2'},
 {'activation': 'leaky',
  'batch_normalize': '1',
  'filters': '64',
  'name': 'convolutional',
  'pad': '1',
  'size': '1',
  'stride': '1'},
 {'activation': 'leaky',
  'batch_normalize': '1',
  'filters': '128',
  '